In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# MyRealty
from ConcreteScrapers.MyRealty.MyRealtyScrapingPipeline import MyRealtyScrapingPipeline

# Storage
from CSVStorage import CSVStorage

In [29]:
storage = CSVStorage("apartments.csv")

In [30]:
scraper_pipeline = MyRealtyScrapingPipeline(
    "https://myrealty.am/en/apartments-for-sale/7784", 
    storage
)

In [33]:
links = scraper_pipeline.get_apartment_links()
for link in links:
    scraper_pipeline.scrape_apartment(link)

Task exception was never retrieved
future: <Task finished name='Task-38' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=ClientOSError(32, 'Broken pipe')>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 113, in download_image
    async with session.get(url) as response:
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 560, in _request
    await resp.start(conn)
  File "/Users/petrostepoyan/opt/anaconda

Task exception was never retrieved
future: <Task finished name='Task-20' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=ClientOSError(32, 'Broken pipe')>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 113, in download_image
    async with session.get(url) as response:
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 560, in _request
    await resp.start(conn)
  File "/Users/petrostepoyan/opt/anaconda

Task exception was never retrieved
future: <Task finished name='Task-1543' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 113, in download_image
    async with session.get(url) as response:
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 560, in _request
    await resp.start(conn)
  File "/Users/petrostepo

Task exception was never retrieved
future: <Task finished name='Task-1558' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=ClientOSError(54, 'Connection reset by peer')>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 113, in download_image
    async with session.get(url) as response:
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client.py", line 560, in _request
    await resp.start(conn)
  File "/Users/petrostepoy

Task exception was never retrieved
future: <Task finished name='Task-1540' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=ClientConnectorError(ConnectionKey(host='myrealty.am', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None), OSError(51, 'Network is unreachable'))>
Traceback (most recent call last):
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/connector.py", line 980, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)  # type: ignore[return-value]  # noqa
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/asyncio/base_events.py", line 1065, in create_connection
    raise exceptions[0]
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/asyncio/base_events.py", line 1050, in create_connection
    sock = await self._connect_s

...

Task exception was never retrieved
future: <Task finished name='Task-23' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 127, in download_image
    f.write(await response.read())
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 1037, in read
    self._body = await self.content.read()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/streams.py", line 375, in read
    block = await self.readany()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9

Task exception was never retrieved
future: <Task finished name='Task-14' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 127, in download_image
    f.write(await response.read())
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 1037, in read
    self._body = await self.content.read()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/streams.py", line 375, in read
    block = await self.readany()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9

Task exception was never retrieved
future: <Task finished name='Task-35' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 127, in download_image
    f.write(await response.read())
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 1037, in read
    self._body = await self.content.read()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/streams.py", line 375, in read
    block = await self.readany()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9

Task exception was never retrieved
future: <Task finished name='Task-1533' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 127, in download_image
    f.write(await response.read())
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 1037, in read
    self._body = await self.content.read()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/streams.py", line 375, in read
    block = await self.readany()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3

Task exception was never retrieved
future: <Task finished name='Task-1532' coro=<MyRealtyApartmentScraper.__scrape_images.<locals>.main() done, defined at /Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py:130> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 133, in main
    await asyncio.gather(*tasks)
  File "/Users/petrostepoyan/Desktop/capstone.tmp/ConcreteScrapers/MyRealty/MyRealtyApartmentScraper.py", line 127, in download_image
    f.write(await response.read())
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 1037, in read
    self._body = await self.content.read()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3.9/site-packages/aiohttp/streams.py", line 375, in read
    block = await self.readany()
  File "/Users/petrostepoyan/opt/anaconda3/lib/python3

In [ ]:
url = "https://bars.am/en/properties/standard/apartment/53896"